# Automated ML

Import all dependencies.

In [1]:
import json
import azureml.core
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initialize workspace

Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136710
aml-quickstarts-136710
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21


## Create an Azure ML experiment

In [3]:
# choose a name for experiment
experiment_name = 'automl-heart-experiment'

experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

## Create a Compute Cluster

In [4]:
amlcompute_cluster_name = "heart-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded...................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
For this project, I used the [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) from Kaggle. 

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

In [5]:
key = "heart-failure"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


### Explore

In [6]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration


Let me explain the reasoning behind my choices for the AutoML Config:
* **experiment_timeout_minutes**: I chose 15 minutes as the maximum amount of time the experiment can takee before it terminates because I have a small dataset with only 299 entries.
* **max_concurrent_iterations**: Represents the maximum number of iterations that would be executed in parallel. The default value is 1.
* **n_cross_validations**: To avoid overfitting, we need to user cross validation.
* **primary_metric**: Area Under Curve (AUC) is one of the most widely used metrics for evaluation. It is used for binary classification problem.
* **task**: Classification, since we want to have a binary prediction (O or 1).

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 15,
    "iterations": 40,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 3,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on heart-compute with default configuration
Running on remote compute: heart-compute
Parent Run ID: AutoML_9245f821-c61e-417e-a3b7-85db6aa64f61

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing value

## Run Details
Use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_9245f821-c61e-417e-a3b7-85db6aa64f61',
 'target': 'heart-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-31T11:50:20.059746Z',
 'endTimeUtc': '2021-01-31T12:13:57.239828Z',
 'properties': {'num_iterations': '40',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'heart-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-heart-experiment","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-136710","workspace_name":"quick-starts-ws-136710","region":"southcentralus","compute_target":"heart-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":40,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_clas

## Best Model


In [11]:
best_run, fitted_model = remote_run.get_output()

In [12]:
best_metrics = best_run.get_metrics()

print("Best Run Id : {}".format(best_run.id), 
      "Accuracy : {}".format(best_metrics['accuracy']), 
      "Best metrics : {}".format(best_metrics), 
      "Best model : {}".format(fitted_model), sep = '\n')

Best Run Id : AutoML_9245f821-c61e-417e-a3b7-85db6aa64f61_36
Accuracy : 0.8595286195286196
Best metrics : {'accuracy': 0.8595286195286196, 'recall_score_micro': 0.8595286195286196, 'precision_score_macro': 0.856624582583786, 'recall_score_weighted': 0.8595286195286196, 'average_precision_score_weighted': 0.9075481993186054, 'f1_score_micro': 0.8595286195286196, 'recall_score_macro': 0.8213546058373645, 'matthews_correlation': 0.6757879301394585, 'precision_score_weighted': 0.8633436421194771, 'AUC_micro': 0.9081776825493998, 'precision_score_micro': 0.8595286195286196, 'balanced_accuracy': 0.8213546058373645, 'AUC_weighted': 0.9002065612410438, 'f1_score_macro': 0.8297084365390162, 'log_loss': 0.4049952940035509, 'weighted_accuracy': 0.8897873448195064, 'average_precision_score_macro': 0.8836335619163993, 'f1_score_weighted': 0.8537289281396695, 'AUC_macro': 0.900206561241044, 'norm_macro_recall': 0.6427092116747289, 'average_precision_score_micro': 0.9087606349983991, 'confusion_matri

In [13]:
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('14',
                                           Pipeline(memory=None,
                                                    steps=[('robustscaler',
                                                            RobustScaler(copy=True,
                                                                         quantile_range=[25,
                                                                                         75],
                                                                         with_centering=True,
                                                                         with_scaling=False)),
                                                           ('extratreesclassifier',
                                                            ExtraTreesClassifier(bootstrap=False,
                                                                                 ccp_alpha=0.0,
              

In [30]:
# Save the best model
model = best_run.register_model(model_name = "heart-failure-best-model-automl", model_path = './outputs/model.pkl')
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-136710', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-136710'), name=heart-failure-best-model-automl, id=heart-failure-best-model-automl:2, version=2, tags={}, properties={})


In [31]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)

heart-failure-best-model-automl version: 2
heart-failure-best-model-automl version: 1
heart-failure-best-model-hdr version: 1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In [41]:
#Save the best model for the deployement
import os
os.makedirs('./aml-model', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./aml-model','heart-failure-best-model-automl.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./aml-model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [42]:
model=best_run.register_model(
            model_name = 'heart-failure-best-model-automl', 
            model_path = './outputs/model.pkl'
)

### Define the environement

In [43]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
environment = Environment.from_conda_specification(name = 'heart-failure-env', file_path = 'conda_env.yml')

### Download the scoring file

In [48]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

### Inference Configuration

In [49]:
inference_config = InferenceConfig(entry_script= 'score.py', environment=environment)

### ACI Configuration

In [50]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

### Deploy

In [51]:
webservice = Model.deploy(workspace=ws, 
                        name='heart-failure-ws', 
                        models=[model], 
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)

In [52]:
webservice

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-136710', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-136710'), name=heart-failure-ws, image_id=None, compute_type=None, state=ACI, scoring_uri=Failed, tags=None, properties=None, created_by={})

In [53]:
# wait for deployment to finish and display the scoring uri and swagger uri
webservice.wait_for_deployment(show_output=True)

print("Status : {}".format(webservice.state), 
      "scoring_uri : {}".format(webservice.scoring_uri), 
      "swagger_uri : {}".format(webservice.swagger_uri), sep = '\n')

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
SucceededACI service creation operation finished, operation "Succeeded"
Status : Healthy
scoring_uri : http://1d45b776-9882-4b8c-991f-f0b9063efca3.southcentralus.azurecontainer.io/score
swagger_uri : http://1d45b776-9882-4b8c-991f-f0b9063efca3.southcentralus.azurecontainer.io/swagger.json


### Consume 
# NOT FINISHED

Send a request to the web service you deployed to test it.

In [56]:
from pprint import pprint

# select samples from the dataframe
test_data = df.sample(5)
test_label = test_data.pop('DEATH_EVENT')

# convert the records to a json data file
data_json = test_data.to_dict(orient='records')

data = json.dumps({'data': data_json})
pprint(data)

('{"data": [{"age": 70.0, "anaemia": 0, "creatinine_phosphokinase": 582, '
 '"diabetes": 0, "ejection_fraction": 40, "high_blood_pressure": 0, '
 '"platelets": 51000.0, "serum_creatinine": 2.7, "serum_sodium": 136, "sex": '
 '1, "smoking": 1, "time": 250}, {"age": 62.0, "anaemia": 1, '
 '"creatinine_phosphokinase": 655, "diabetes": 0, "ejection_fraction": 40, '
 '"high_blood_pressure": 0, "platelets": 283000.0, "serum_creatinine": 0.7, '
 '"serum_sodium": 133, "sex": 0, "smoking": 0, "time": 233}, {"age": 65.0, '
 '"anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 1, '
 '"ejection_fraction": 40, "high_blood_pressure": 0, "platelets": 270000.0, '
 '"serum_creatinine": 1.0, "serum_sodium": 138, "sex": 0, "smoking": 0, '
 '"time": 140}, {"age": 42.0, "anaemia": 0, "creatinine_phosphokinase": 5209, '
 '"diabetes": 0, "ejection_fraction": 30, "high_blood_pressure": 0, '
 '"platelets": 226000.0, "serum_creatinine": 1.0, "serum_sodium": 140, "sex": '
 '1, "smoking": 1, "time": 87}, {

In [57]:
# get inference
output = webservice.run(data)
print(output)

{"result": [0, 0, 0, 0, 0]}


Print the logs of the web service and delete the service

In [58]:
webservice.get_logs()

'2021-01-31T12:59:59,063216479+00:00 - iot-server/run \n2021-01-31T12:59:59,065148396+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-01-31T12:59:59,067241222+00:00 - gunicorn/run \n2021-01-31T12:59:59,067842859+00:00 - rsyslog/run 

In [65]:
# select samples from the dataframe
test_data = df.sample(10)
test_label = test_data.pop('DEATH_EVENT')

# convert the records to a json data file
data_json = test_data.to_dict(orient='records')

data = json.dumps({'data': data_json})

# get inference
output = webservice.run(data)
print(output)

{"result": [1, 0, 1, 1, 0, 1, 0, 0, 0, 0]}


In [66]:
webservice.get_logs()

'2021-01-31T12:59:59,063216479+00:00 - iot-server/run \n2021-01-31T12:59:59,065148396+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-01-31T12:59:59,067241222+00:00 - gunicorn/run \n2021-01-31T12:59:59,067842859+00:00 - rsyslog/run 

In [67]:
webservice.delete()
compute_target.delete()